## 抓到Footnotes <br>
+ 目前會抓到帶有 " * " 的FN 

+ Counsel: 96個 **p.484出現在FN裡面**
+ Judges: 94個 (P471沒有)
+ Opinion by: 95個

+ 有些會抓到Opinion內容

In [1]:
# Explain
import fitz
import re
import csv

pdf_file_path = r'C:\Users\User\Dropbox\textmining1\PDF\Files (100).PDF'
csv_file_path = r'C:\Users\User\Dropbox\textmining1\opinion\opinion_100_test.csv'


pattern = r'(Counsel:.*?|Judges:.*?(?=Opinion by:)|Opinion by:.*?)((?=Counsel:)|(?=Judges:)|(?=Opinion by:)|(?=Opinion)|$)'

# " (Counsel:.*?|Judges:.*?(?=Opinion by:)|Opinion by:.*?) "  :
# capture content of " Counsel: ", " Judges: " 到 "Opinion by:" or "Opinion"

# " ((?=Counsel:)|(?=Judges:)|(?=Opinion by:)|(?=Opinion)|$) " :
# capture the type of the match ("Counsel:", "Judges:", "Opinion by:", "Opinion", or the end of the string).


def find_page_num(text, match_start, page_delimiters):

#找出匹配文本再PDF的頁碼. Text(整個PDF文本內容) . match_start(當前匹配文本起始位置)
# page_delimiters(包含每頁結尾在text中的列表) 將每頁文本加入all_text並記錄長度, 每個元素代表文本結束的位置
    for i, delimiter in enumerate(page_delimiters): # i為當前頁碼索引 從0開始
        if match_start < delimiter: #檢查match_start是否小於delimiter  , 小於代表匹配文本位於這一頁或之前的頁面
            return i #頁碼通常從1開始, 所以後續都會對這個索引 + 1
    return len(page_delimiters) # 表示匹配文本位於文檔最後一頁

# 加入計數器
counsel_count = 0
judges_count = 0
opinion_by_count = 0

with fitz.open(pdf_file_path) as doc:
    all_text = "" #儲存整個文本內容
    page_delimiters = [] #儲存每頁文本結束的列表

    for page_num in range(len(doc)): #閱讀文本每一頁 . len(doc):總頁數
        page = doc.load_page(page_num) #載入當前頁
        page_text = page.get_text("text")  # 獲取當前頁的文本
        all_text += page.get_text("text") + "\f" #從當前頁提取文本並添加到all_text中 # 使用分頁符號 "\f" 作為分頁標記
        page_delimiters.append(len(all_text))  # 在page_delimiters列表添加累積文本長度,也就是當前頁結束位置, 紀錄每頁結束的位置.
        # 在每段文本前加上頁碼標記
        formatted_page_text = f'頁碼 {page_num + 1}:\n{page_text}'
    # 移除包含 "Page \d+ of \d+" 的字樣
    all_text = re.sub(r'Page \d+ of \d+', '', all_text)

# re.sub 查找和替換 符合正則表達式的字串, ''表示將匹配內容刪除
# \d+ 數字 . Page和of 為單詞 . 表 刪除 ex Page 23 of 456 將其中all_text中刪除

    # 移除其他不重要的字樣
    all_text = re.sub(r'\d+ Cal\..*?\*{1,4}\d+; (?:\d{4} )?Cal\..*?\*{1,4}\d+', '', all_text)

# Cal\. = Cal. " .*? " 非貪婪匹配任所有字符直到遇到後面的模式 "\*{1,4}" 匹配1~4個'*' ;匹配分號
# (?:\d{4} ) 匹配四位數字後跟一個空格 , Cal\..*?\*{1,4}\d+ 同上 匹配Cal. 後面的文字,星號,數字

    with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
      #確保文件使用utf-8編碼, newline=''為了避免在不同操作系統寫入時產生新行問題
        csv_writer = csv.writer(csv_file) # 創建CSV寫入器,將數據輸入
        csv_writer.writerow(['頁碼', '內容類型', '內容']) #將標題寫入

        matches = re.finditer(pattern, all_text, re.DOTALL) # find all occourance of pattern in 'text'
        for match in matches: #loop initiate matches from re.finditer
            match_content = match.group(1).strip() # 提取第一個匹配組內容,去除首尾空格
            match_type = match.group(2).strip()  # 提取第二個匹配組內容,去除首尾空格
            match_page = find_page_num(all_text, match.start(), page_delimiters)
            # 調用find_page_num函數確定匹配內容所在頁碼,將這些資訊寫入CSV

            if 'Counsel:' in match_content:
                csv_writer.writerow([match_page + 1, 'Counsel', match_content])
                counsel_count += 1
            elif 'Judges:' in match_content:
                csv_writer.writerow([match_page + 1, 'Judges', match_content])
                judges_count += 1
            elif 'Opinion by:' in match_content:
                csv_writer.writerow([match_page + 1, 'Opinion by', match_content])
                opinion_by_count += 1

print(f"完成：{csv_file_path}")
print(f"Counsel 數量: {counsel_count}")
print(f"Judges 數量: {judges_count}")
print(f"Opinion by 數量: {opinion_by_count}")

完成：C:\Users\User\Dropbox\textmining1\opinion\opinion_100_test.csv
Counsel 數量: 95
Judges 數量: 92
Opinion by 數量: 95


## 最終版Code

In [2]:
import fitz  # PyMuPDF
import re
import csv

pdf_file_path = r"C:\Users\User\Dropbox\textmining1\PDF\Files (100).PDF"
csv_file_path = r"C:\Users\User\Dropbox\textmining1\opinion\opinion_100_test.csv"

# 更新後的正則表達式，確保遇到下一個標記或"Opinion"時停止
pattern = r'(Counsel:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)|(Judges:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)|(Opinion by:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)'

def find_page_num(text, match_start, page_delimiters):
    for i, delimiter in enumerate(page_delimiters):
        if match_start < delimiter:
            return i
    return len(page_delimiters)

counsel_count = 0
judges_count = 0
opinion_by_count = 0

with fitz.open(pdf_file_path) as doc:
    all_text = ""
    page_delimiters = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        page_text = page.get_text("text")
        all_text += page_text + "\f"
        page_delimiters.append(len(all_text))

    # 移除頁碼和案例引用
    all_text = re.sub(r'Page \d+ of \d+', '', all_text)
    all_text = re.sub(r'\d+ Cal\..*?\*{1,4}\d+; (?:\d{4} )?Cal\..*?\*{1,4}\d+', '', all_text)

    with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['頁碼', '內容類型', '內容'])

        matches = re.finditer(pattern, all_text, re.DOTALL)
        for match in matches:
            for group_num in range(1, 4):
                if match.group(group_num):
                    match_content = match.group(group_num).strip()
                    match_page = find_page_num(all_text, match.start(group_num), page_delimiters)
                    content_type = 'Counsel' if group_num == 1 else 'Judges' if group_num == 2 else 'Opinion by'
                    csv_writer.writerow([match_page + 1, content_type, match_content])
                    if content_type == 'Counsel':
                        counsel_count += 1
                    elif content_type == 'Judges':
                        judges_count += 1
                    elif content_type == 'Opinion by':
                        opinion_by_count += 1
                    break

print(f"完成：{csv_file_path}")
print(f"Counsel 數量: {counsel_count}")
print(f"Judges 數量: {judges_count}")
print(f"Opinion by 數量: {opinion_by_count}")


完成：C:\Users\User\Dropbox\textmining1\opinion\opinion_100_test.csv
Counsel 數量: 95
Judges 數量: 94
Opinion by 數量: 95


## 遍歷資料夾及子資料夾PDF
+ 匯入一個CSV 但 有PDF檔案名稱

In [2]:
import fitz  # PyMuPDF
import re
import csv
import os

# 指定需要遍歷的根資料夾路徑，這裡將其設置為Data 1資料夾的路徑
root_folder_path = r"C:\Users\User\Dropbox\textmining1\Data 1"
# 指定CSV文件路徑
csv_file_path = r"C:\Users\User\Dropbox\textmining1\opinion\opinion_summary.csv"

# 正則表達式
pattern = r'(Counsel:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)|(Judges:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)|(Opinion by:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)'

def find_page_num(text, match_start, page_delimiters):
    for i, delimiter in enumerate(page_delimiters):
        if match_start < delimiter:
            return i
    return len(page_delimiters)

def process_pdf_files(folder_path, csv_writer):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".pdf"):  # 支援小寫或大寫的PDF檔案副檔名
                pdf_file_path = os.path.join(root, file)
                process_single_pdf(pdf_file_path, csv_writer)

def process_single_pdf(pdf_file_path, csv_writer):
    with fitz.open(pdf_file_path) as doc:
        all_text = ""
        page_delimiters = []

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            page_text = page.get_text("text")
            all_text += page_text + "\f"
            page_delimiters.append(len(all_text))

        all_text = re.sub(r'Page \d+ of \d+', '', all_text)
        all_text = re.sub(r'\d+ Cal\..*?\*{1,4}\d+; (?:\d{4} )?Cal\..*?\*{1,4}\d+', '', all_text)

        matches = re.finditer(pattern, all_text, re.DOTALL)
        for match in matches:
            for group_num in range(1, 4):
                if match.group(group_num):
                    match_content = match.group(group_num).strip()
                    match_page = find_page_num(all_text, match.start(group_num), page_delimiters)
                    content_type = 'Counsel' if group_num == 1 else 'Judges' if group_num == 2 else 'Opinion by'
                    csv_writer.writerow([os.path.basename(pdf_file_path), match_page + 1, content_type, match_content])

with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['文件名稱', '頁碼', '內容類型', '內容'])
    process_pdf_files(root_folder_path, csv_writer)

print(f"完成：{csv_file_path}")


完成：C:\Users\User\Dropbox\textmining1\opinion\opinion_summary.csv


## 一個PDF一個CSV

In [8]:
import fitz  # PyMuPDF
import re
import csv
import os
from pathlib import Path

# 更新後的正則表達式，確保遇到下一個標記或"Opinion"時停止
pattern = r'(Counsel:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)|(Judges:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)|(Opinion by:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)'

def find_page_num(text, match_start, page_delimiters):
    for i, delimiter in enumerate(page_delimiters):
        if match_start < delimiter:
            return i
    return len(page_delimiters)

def process_single_pdf(pdf_file_path, csv_file_path):
    with fitz.open(pdf_file_path) as doc:
        all_text = ""
        page_delimiters = []

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            page_text = page.get_text("text")
            all_text += page_text + "\f"
            page_delimiters.append(len(all_text))

        # 移除頁碼和案例引用
        all_text = re.sub(r'Page \d+ of \d+', '', all_text)
        all_text = re.sub(r'\d+ Cal\..*?\*{1,4}\d+; (?:\d{4} )?Cal\..*?\*{1,4}\d+', '', all_text)

        with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['頁碼', '內容類型', '內容'])

            matches = re.finditer(pattern, all_text, re.DOTALL)
            for match in matches:
                for group_num in range(1, 4):
                    if match.group(group_num):
                        match_content = match.group(group_num).strip()
                        match_page = find_page_num(all_text, match.start(group_num), page_delimiters)
                        content_type = 'Counsel' if group_num == 1 else 'Judges' if group_num == 2 else 'Opinion by'
                        csv_writer.writerow([match_page + 1, content_type, match_content])
                        break

def process_pdf_files(folder_path, output_folder_path):
    # 确保输出文件夹存在
    Path(output_folder_path).mkdir(parents=True, exist_ok=True)

    # 初始化子文件夹标签索引
    label_index = 0  # 从0开始，对应于'a'

    # 定义子文件夹的标识符列表
    subfolder_labels = list('abcdefghijklmnopqrstuvwxyz')
    
    # 明确指定顶层文件夹的处理顺序，如果默认排序不适用
    top_folders = [r'/Users/tangjiahong/Dropbox/textmining1/Data 1', r'/Users/tangjiahong/Dropbox/textmining1/Data 2', r'/Users/tangjiahong/Dropbox/textmining1/Data 3']  # 根据需要调整

    for top_folder in top_folders:
        top_folder_path = os.path.join(folder_path, top_folder)

        # 获取子文件夹并按预期的顺序排序
        subdirs = sorted([d for d in os.listdir(top_folder_path) if os.path.isdir(os.path.join(top_folder_path, d))])
        
        for subdir in subdirs:
            subdir_path = os.path.join(top_folder_path, subdir)

            # 获取PDF文件并按字母顺序排序
            pdf_files = sorted([f for f in os.listdir(subdir_path) if f.lower().endswith('.pdf')])
            
            for file_index, file in enumerate(pdf_files, start=1):
                pdf_file_path = os.path.join(subdir_path, file)

                # 使用子文件夹标签和文件编号生成CSV文件名
                letter = subfolder_labels[label_index % len(subfolder_labels)]
                pdf_label = f"{letter}{file_index}_opinion.csv"
                csv_file_path = os.path.join(output_folder_path, pdf_label)
                process_single_pdf(pdf_file_path, csv_file_path)
                
            # 处理完一个子文件夹后，更新标签索引
            label_index += 1

root_folder_path = "/Users/tangjiahong/Dropbox/textmining1"
output_folder_path = "/Users/tangjiahong/Dropbox/textmining1/Opinion"

process_pdf_files(root_folder_path, output_folder_path)


## 生成一個整合CSV , 照英文字母排序

In [11]:
import csv
import os
from pathlib import Path
import re
import sys

def natural_sort_key(s):
    """
    提取用于自然排序的键（包括数字排序）。
    """
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

def merge_csv_files(input_folder, output_csv_path):
    # 确保输出文件夹存在
    Path(output_csv_path).parent.mkdir(parents=True, exist_ok=True)
    
    # 尝试增加字段大小限制
    csv.field_size_limit(sys.maxsize)

    with open(output_csv_path, 'w', newline='', encoding='utf-8') as output_file:
        output_writer = csv.writer(output_file)
        headers_written = False

        # 使用自定义的自然排序键来排序文件名
        for input_file in sorted(os.listdir(input_folder), key=natural_sort_key):
            if input_file.endswith('.csv'):
                pdf_label = input_file.split('_')[0]  # 假设文件名格式为 'd1_FN.csv'
                with open(Path(input_folder) / input_file, newline='', encoding='utf-8') as csv_file:
                    csv_reader = csv.reader(csv_file)
                    headers = next(csv_reader)

                    if not headers_written:
                        output_writer.writerow(['PDF'] + headers)
                        headers_written = True

                    for row in csv_reader:
                        output_writer.writerow([pdf_label] + row)

input_folder = r'/Users/tangjiahong/Dropbox/textmining1/Opinion'
output_csv_path = r'/Users/tangjiahong/Dropbox/textmining1/merge_CSV/merge_Opinion.csv'

merge_csv_files(input_folder, output_csv_path)


+ 0722(6).pdf p.1090 Counsel ~ p.1203 多達 100多頁